# control_iSarcasm
This notebook trains our control model--a model that is only fine-tuned on our target task, iSarcasm.

## Imports & Settings

First, update working directory to parent so that we may use our custom functions

In [1]:
import os
os.chdir('..')
# os.getcwd( )

In [2]:
import params
from utils import *
from trainer import *

import numpy as np
import pandas as pd
from datasets import load_from_disk

from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

# suppress model warning
from transformers import logging
logging.set_verbosity_error()

# set logging level
import logging
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [3]:
# set general seeds
set_seeds(1)

# set dataloader generator seed
g = torch.Generator()
g.manual_seed(1)

# set params for this model
params.num_labels = 2
params.output_dir = "model_saves/control_iSarcasm_03"
params.dataset_path = "data/target_iSarcasmEval/itesd_iSarcasmEval_balanced.hf"

# Ensure we're on an ARM environment if necessary.
platform_check()

We're Armed: macOS-13.1-arm64-i386-64bit


## Load Data

### iSarcasm

In [4]:
datasets = load_from_disk(params.dataset_path)
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4266
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 628
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1400
    })
})

In [5]:
# we will need to view and prep the datasets
# this is more easily done as dataframes
train_df = datasets['train'].to_pandas()
validate_df = datasets['validation'].to_pandas() 

In [6]:
# view training dataset
print("train_df Info:")
print(train_df.info())
print("\ntrain_df Value Counts")
print(train_df['label'].value_counts())

train_df Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4266 entries, 0 to 4265
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4266 non-null   object
 1   label   4266 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 66.8+ KB
None

train_df Value Counts
1    2133
0    2133
Name: label, dtype: int64


In [7]:
# view validation dataset
print("validate_df Info:")
print(validate_df.info())
print("\n validate_df Value Counts")
print(validate_df['label'].value_counts())

validate_df Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 628 entries, 0 to 627
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    628 non-null    object
 1   label   628 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.9+ KB
None

 validate_df Value Counts
0    467
1    161
Name: label, dtype: int64


## Preprocess

In [8]:
train_token_ids, train_attention_masks = encode_text(train_df.text.values)
validate_token_ids, validate_attention_masks = encode_text(validate_df.text.values)

In [9]:
train_features = []
for i in range(len(train_token_ids)):
    train_features.append({'label': train_df.label.values[i], 
                           'input_ids': train_token_ids[i], 
                           'attention_mask':train_attention_masks[i]})

validate_features = []
for i in range(len(validate_token_ids)):
    validate_features.append({'label': validate_df.label.values[i], 
                              'input_ids': validate_token_ids[i],
                              'attention_mask':validate_attention_masks[i]})

## Data Split
We split the dataset into train (80%) and validation (20%) sets, and wrap them around a torch.utils.data.DataLoader object.

In [10]:
# Prepare DataLoader
train_dataloader = DataLoader(
            train_features,
            sampler = RandomSampler(train_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=collate
        )

validation_dataloader = DataLoader(
            validate_features,
            sampler = RandomSampler(validate_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=collate
        )

In [11]:
# view an example from the dataloader
next(iter(train_dataloader))

[tensor([[    0,  1039,   119,  1176,  4244, 16475, 24661,   101,    10,   670,
          12122,    13, 42007,   975,     2,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1],
         [    0, 17693,   141,  7340,   939, 31312,  2185,    15,  7011,    66,
            141,  5939,  1577,     2,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1],
         [    0,  1213,   269,   362,   964,   160,  3909, 17841, 12410, 18636,
          15375,  6569, 1047

## Train

Download transformers.RobertaForSequenceClassificatio, which is a RoBERTa model with a linear layer for sentence classification (or regression) on top of the pooled output:

In [12]:
# Load the RobertaForSequenceClassification model
model = RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                         num_labels = params.num_labels,
                                                         output_attentions = False,
                                                         output_hidden_states = False,
                                                         )

from torchinfo import summary
summary(model, input_size=(1, 512), dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                       Output Shape              Param #
RobertaForSequenceClassification                             [1, 2]                    --
├─RobertaModel: 1-1                                          [1, 512, 768]             --
│    └─RobertaEmbeddings: 2-1                                [1, 512, 768]             --
│    │    └─Embedding: 3-1                                   [1, 512, 768]             38,603,520
│    │    └─Embedding: 3-2                                   [1, 512, 768]             768
│    │    └─Embedding: 3-3                                   [1, 512, 768]             394,752
│    │    └─LayerNorm: 3-4                                   [1, 512, 768]             1,536
│    │    └─Dropout: 3-5                                     [1, 512, 768]             --
│    └─RobertaEncoder: 2-2                                   [1, 512, 768]             --
│    │    └─ModuleList: 3-6                                  --               

Set model to device, initialize trainer

In [13]:
model.to(params.device)
print(f"Device: {params.device}")

optimizer = torch.optim.Adam(params=model.parameters(), 
                             lr=params.learning_rate,
                             weight_decay=params.weight_decay) #roberta

trainer = Trainer(model=model,
                  device=params.device,
                  tokenizer=params.tokenizer,
                  train_dataloader=train_dataloader,
                  validation_dataloader=validation_dataloader,
                  epochs=params.epochs,
                  optimizer=optimizer,
                  val_loss_fn=params.val_loss_fn,
                  num_labels=params.num_labels,
                  output_dir=params.output_dir,
                  save_freq=params.save_freq,
                  checkpoint_freq=params.checkpoint_freq)

output_parameters()

Device: mps

          Training Dataset: data/target_iSarcasmEval/itesd_iSarcasmEval_balanced.hf
          Number of Labels: 2
          Batch Size: 16
          Learning Rate: 1e-05
          Weight Decay: 0
          Epochs: 10
          Output Directory: model_saves/control_iSarcasm_03
          Save Frequency: 1
          Checkpoint Frequency: 1
          Max Length: 256
          


Fit the model to our training data.

In [14]:
trainer.fit()

	 Validation 39: 100%|██████████| 40/40 [00:07<00:00,  5.14batch/s]

 	 - Train loss: 0.666084
	 - Validation Loss: 0.483086
	 - Validation Accuracy: 0.765625
	 - Validation F1: 0.498936
	 - Validation Recall: 0.499643
	 - Validation Precision: 0.577500 

	 * Model @ epoch 1 saved to model_saves/control_iSarcasm_03/E01_A0.77_F0.5
	 * Model checkpoint saved to model_saves/control_iSarcasm_03/E01_A0.77_F0.5/checkpoint.pt

	 Validation 39: 100%|██████████| 40/40 [00:05<00:00,  7.05batch/s]

 	 - Train loss: 0.388412
	 - Validation Loss: 0.632560
	 - Validation Accuracy: 0.782813
	 - Validation F1: 0.335556
	 - Validation Recall: 0.281161
	 - Validation Precision: 0.522917 

	 * Model @ epoch 2 saved to model_saves/control_iSarcasm_03/E02_A0.78_F0.34
	 * Model checkpoint saved to model_saves/control_iSarcasm_03/E02_A0.78_F0.34/checkpoint.pt

	 Validation 39: 100%|██████████| 40/40 [00:06<00:00,  6.03batch/s]

 	 - Train loss: 0.133542
	 - Validation Loss: 0.881882
	 - Validation Accuracy: 